## An Overview of the Machine Learning Life Cycle

### The ML life cycle in practice

Problem statement
- customer segments and customer lifetime value (LTV)

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data/OnlineRetail.csv", encoding="unicode_escape")

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


• InvoiceNo: 고유 주문 ID(데이터는 정수 유형)

• StockCode: 제품의 고유 ID, 데이터는 문자열 유형입니다.

• Description: 제품의 고유 ID, 데이터는 문자열 유형입니다.

• Quantity: 주문한 제품의 단위 수입니다.

• InvoiceDate: The date when the invoice was generated

• UnitPrice: 인보이스가 생성된 날짜입니다.

• CustomerID: 제품을 주문한 고객의 고유 ID입니다.

• Country: The country where the product was ordered

이 연습에서는 월별 수익을 계산하고 계절성을 살펴보겠습니다. 

다음 코드 블록은 InvoiceDate 열에서 연도 및 월(yyyymm) 정보를 추출하고, 단위 가격 및 수량 열을 곱하여 각 거래의 수익 속성을 계산한 다음, 추출된 연도-월(yyyymm) 열을 기준으로 수익을 집계합니다.앞의 코드 문에서 계속해 보겠습니다:

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors = 'coerce')
data['yyyymm']=data['InvoiceDate'].dt.strftime('%Y%m')
data['revenue'] = data['UnitPrice'] * data['Quantity']

revenue_df = data.groupby(['yyyymm'])['revenue'].sum().reset_index()
revenue_df.head()

,yyyymm,revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121


In [5]:
import plotly.express as px

In [6]:
revenue_df = revenue_df.sort_values(by='yyyymm')

In [7]:
fig = px.bar(revenue_df, x="yyyymm", y="revenue",
title="Monthly Revenue")
fig.update_xaxes(type='category')
fig.show()

월별 active cutomer 계산

In [8]:
active_customer_df = data.groupby(['yyyymm'])['CustomerID'].nunique().reset_index()
active_customer_df.columns = ['yyyymm','No of Active customers']
active_customer_df.head()

,yyyymm,No of Active customers
0,201012,948
1,201101,783
2,201102,798
3,201103,1020
4,201104,899


In [9]:
fig = px.bar(active_customer_df, x="yyyymm", y="No of Active customers",
title="Monthly Active customers")
fig.update_xaxes(type='category')
fig.show()

Model

고객 생애 가치(CLTV)는 고객과 기업의 라이프사이클과 관련된 순이익으로 정의됩니다. 

간단히 말해, CLV/LTV는 각 고객의 비즈니스 가치에 대한 예측입니다.

생애 가치를 예측하는 방법에는 여러 가지가 있습니다. 

하나는 회귀 문제인 고객의 가치를 예측하는 방법이고, 다른 하나는 분류 문제인 고객 그룹을 예측하는 방법일 수 있습니다. 

이 연습에서는 후자의 접근 방식을 사용합니다.

For this exercise, we will segment customers into the following groups:  
• Low LTV: Less active or low revenue customers  
• Mid-LTV: Fairly active and moderate revenue customers  
• High LTV: High revenue customers – the segment that we don't want to lose

3개월 분량의 데이터를 사용하여 고객의 최근성(R), 빈도(F), 금액(M) 지표를 계산하여 기능을 생성할 것입니다.   

이러한 기능을 확보한 후에는 6개월 분량의 데이터를 사용하여 모든 고객의 수익을 계산하고 LTV 클러스터 라벨(낮은 LTV, 중간 LTV, 높은 LTV)을 생성할 것입니다.   

그런 다음 생성된 레이블과 기능을 사용하여 신규 고객 그룹을 예측하는 데 사용할 수 있는 XGBoost 모델을 훈련합니다.

Feature engineering